### Get Trusts

In [1]:
import arcpy
import os
import sys
ws = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\TaxLots\TaxLots.gdb'
arcpy.env.workspace = ws

In [2]:
def unique_values(table, field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
    
        return sorted({row[0] for row in cursor if row[0] != None})
        del cursor

In [3]:
def sqlClause(termList, fc, fieldName):
    owner_string = ""
    for owner in termList:
        if termList.index(owner) != len(termList) -1:
            owner_string = owner_string + """{field} = '{owner}' Or """.format(field = arcpy.AddFieldDelimiters(fc, fieldName), 
                                                                               owner = owner)
        elif termList.index(owner) == len(termList) -1:
            owner_string = owner_string + """{field} = '{owner}'""".format(field = arcpy.AddFieldDelimiters(fc, fieldName), 
                                                                               owner = owner)
    return owner_string

In [4]:
def removeTerms(OgList, removeList):
    for item in removeList:
        if item in OgList:
            OgList.remove(item)
    return OgList
    

In [48]:
def sqlClause(termList, fieldName):
    owner_string = ""
    for owner in termList:
        if termList.index(owner) != len(termList) -1:
            owner_string = owner_string + f"{fieldName} = '{owner}' Or "
        elif termList.index(owner) == len(termList) -1:
            owner_string = owner_string + f"{fieldName} = '{owner}'"
    return owner_string

In [73]:
owners = unique_values('county_merge', 'Land_Owner')

In [74]:
trustOwners = [i for i in owners if ('LAND TRUST' in i)]

In [75]:
removeFromTrustList = ['HOMELAND TRUST LLC', 'JOHANNES/JEANNETTE L HOOGLAND TRUST', 'JOHN L/SHIRLEY J WENTLAND TRUST', 
                       'LAURIE A ENGLAND TRUST', 'MIDLAND TRUST COMPANY,', 'JONES FAMILY TIMBERLAND TRUST']

In [76]:
trustOwners = removeTerms(trustOwners, removeFromTrustList)

In [77]:
print(trustOwners)

['BROWNSMEAD LAND TRUST', 'COLUMBIA LAND TRUST', 'COLUMBIA LAND TRUST,', 'DAWNED ON ME LAND TRUST', 'FRIENDS OF THE COLUMBIA GORGE LAND TRUST', 'FRIENDS OF THE COLUMBIA RIVER GORGE LAND TRUST', 'GE KALAMA LAND TRUST', 'KINCAID JUANITA M LAND TRUST', 'LAND TRUST #5439 & #5443 SE 120TH', 'OREGON SUSTAINABLE AGRICULTURE LAND TRUST', 'OREGON SUSTAINABLE AGRICULTURE LAND TRUST INC', 'PORTLAND COMMUNITY LAND TRUST', 'PORTLAND COMUNITY LAND TRUST', 'RIVERVIEW LAND TRUST', 'SPORTSMENS NATIONAL LAND TRUST INC,', 'STYNER LAND TRUST,', 'THE 3009 SE 115TH LAND TRUST', 'THE 6306 SE 49TH LAND TRUST', 'THE SAUVIE ISLAND TRUST']


In [71]:
badString = ["SPORTSMEN'S NATIONAL LAND TRUST INC,"]

In [72]:
# Remove ' form any owners
with arcpy.da.UpdateCursor('county_merge', 'Land_Owner') as cursor:
    for row in cursor:
        if row[0] in badString:
           
            row[0] = row[0].replace("'", "")
        cursor.updateRow(row)
    del cursor

In [85]:
sql = sqlClause(trustOwners, 'Land_Owner')

In [86]:
sql

"Land_Owner = 'BROWNSMEAD LAND TRUST' Or Land_Owner = 'COLUMBIA LAND TRUST' Or Land_Owner = 'COLUMBIA LAND TRUST,' Or Land_Owner = 'DAWNED ON ME LAND TRUST' Or Land_Owner = 'FRIENDS OF THE COLUMBIA GORGE LAND TRUST' Or Land_Owner = 'FRIENDS OF THE COLUMBIA RIVER GORGE LAND TRUST' Or Land_Owner = 'GE KALAMA LAND TRUST' Or Land_Owner = 'KINCAID JUANITA M LAND TRUST' Or Land_Owner = 'LAND TRUST #5439 & #5443 SE 120TH' Or Land_Owner = 'OREGON SUSTAINABLE AGRICULTURE LAND TRUST' Or Land_Owner = 'OREGON SUSTAINABLE AGRICULTURE LAND TRUST INC' Or Land_Owner = 'PORTLAND COMMUNITY LAND TRUST' Or Land_Owner = 'PORTLAND COMUNITY LAND TRUST' Or Land_Owner = 'RIVERVIEW LAND TRUST' Or Land_Owner = 'SPORTSMENS NATIONAL LAND TRUST INC,' Or Land_Owner = 'STYNER LAND TRUST,' Or Land_Owner = 'THE 3009 SE 115TH LAND TRUST' Or Land_Owner = 'THE 6306 SE 49TH LAND TRUST' Or Land_Owner = 'THE SAUVIE ISLAND TRUST'"

In [87]:
floodPlain = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\FloodPlain\Hydrogeo_III_merged.shp'

In [93]:
floodSelection = arcpy.SelectLayerByLocation_management('county_merge', 'INTERSECT', floodPlain)
selection = arcpy.SelectLayerByAttribute_management(floodSelection, selection_type = 'SUBSET_SELECTION', where_clause = sql)
arcpy.CopyFeatures_management(selection, 'landTrusts')

<Result 'C:\\Users\\Ricardo\\Desktop\\PCC\\Internship\\UpdatedCountyTaxLots\\TaxLots\\TaxLots.gdb\\landTrusts'>